In [31]:
import pandas as pd
import os
import copy
from collections import Counter
import json
import pprint
import numpy as np

In [38]:
def pairwiseVoteDifferentials(election):
    voteRecord = election[0]
    candidateLookup = election[1]
    numCands = len(candidateLookup)
    voteDifferential = [[0 for i in range(numCands)] for j in range(numCands)]
    for voter in voteRecord:
        ranked = voteRecord[voter]
        for candA in range(1, numCands + 1):
            for candB in range(candA + 1, numCands + 1):
                if (candA in ranked and candB in ranked):
                    if (ranked.index(candA) < ranked.index(candB)):
                        voteDifferential[candA - 1][candB - 1] += 1
                        voteDifferential[candB - 1][candA - 1] -= 1
                    else:
                        voteDifferential[candA - 1][candB - 1] -= 1
                        voteDifferential[candB - 1][candA - 1] += 1
                elif (candA in ranked):
                    voteDifferential[candA - 1][candB - 1] += 1
                    voteDifferential[candB - 1][candA - 1] -= 1
                elif (candB in ranked):
                    voteDifferential[candA - 1][candB - 1] -= 1
                    voteDifferential[candB - 1][candA - 1] += 1
    return voteDifferential

In [48]:
with open('voterecord.json', 'r') as f:
    voteRecord = json.load(f)

In [50]:
candidates = {'257465': "Adrienne E. Adams",
             '254052': "Andrew M. Cuomo",
             '254130': "Brad Lander",
             '254365': "Jessica Ramos",
             '254607': "Michael Blake",
             '258719': "Paperboy Love Prince",
             '258824': "Paperboy Love Prince",
             '254393': "Scott M. Stringer",
             '258821': "Selma K. Bartholomew",
             '257441': "Whitney R. Tilson",
             '255950': "Zellnor Myrie",
             '254286': "Zohran Kwame Mamdani"}
candidateList = ["Adrienne E. Adams",
                 "Andrew M. Cuomo",
                 "Brad Lander",
                 "Jessica Ramos",
                 "Michael Blake",
                 "Paperboy Love Prince",
                 "Scott M. Stringer",
                 "Selma K. Bartholomew",
                 "Whitney R. Tilson",
                 "Zellnor Myrie",
                 "Zohran Kwame Mamdani"]
for vote in voteRecord:
    newRanks = []
    seen = set()
    for i in range(len(voteRecord[vote])):
        curCand = voteRecord[vote][i]
        if (curCand not in ['undervote', 'Write-in', 'overvote']):
            if (curCand not in seen):
                newRanks.append(candidates[curCand])
                seen.add(curCand)
    voteRecord[vote] = newRanks

In [52]:
voteRecordNums = {}
candidateToNum = {"Adrienne E. Adams": 1,
                 "Andrew M. Cuomo": 2,
                 "Brad Lander": 3,
                 "Jessica Ramos": 4,
                 "Michael Blake": 5,
                 "Paperboy Love Prince": 6,
                 "Scott M. Stringer": 7,
                 "Selma K. Bartholomew": 8,
                 "Whitney R. Tilson": 9,
                 "Zellnor Myrie": 10,
                 "Zohran Kwame Mamdani": 11}
numToCandidate = {1: "Adrienne E. Adams",
                  2: "Andrew M. Cuomo",
                  3: "Brad Lander",
                  4: "Jessica Ramos",
                  5: "Michael Blake",
                  6: "Paperboy Love Prince",
                  7: "Scott M. Stringer",
                  8: "Selma K. Bartholomew",
                  9: "Whitney R. Tilson",
                  10: "Zellnor Myrie",
                  11: "Zohran Kwame Mamdani"}
for vote in voteRecord:
    numVote = []
    for i in range(len(voteRecord[vote])):
        numVote.append(candidateToNum[voteRecord[vote][i]])
    voteRecordNums[vote] = numVote

In [ ]:
def twoCandsHeadToHead(cand1, cand2):
    precinctResults = {}
    labelCand1 = str(cand1) + " over " + str(cand2)
    labelCand2 = str(cand2) + " over " + str(cand1)
    labelSum = str(cand1) + ' all ' + str(cand2)
    for vote in voteRecordNums:
        precinctResults[vote[:5]] = {labelCand1: 0, labelCand2: 0}
    for vote in voteRecordNums:
        tally = precinctResults[vote[:5]]
        ranks = voteRecordNums[vote]
        if (cand1 in ranks and cand2 in ranks):
            if (ranks.index(cand1) < ranks.index(cand2)):
                tally[labelCand1] += 1
            else:
                tally[labelCand2] += 1
        elif (cand1 in ranks):
            tally[labelCand1] += 1
        elif (cand2 in ranks):
            tally[labelCand2] += 1
    precinctDf = pd.DataFrame.from_dict(precinctResults, orient='index')
    precinctDf[str(cand1) + "-" + str(cand2)] = np.where(precinctDf[labelCand1] + precinctDf[labelCand2] == 0, 0, (precinctDf[labelCand1] - precinctDf[labelCand2])/(precinctDf[labelCand1] + precinctDf[labelCand2]))
    return precinctDf

In [ ]:
allPrecincts = {}
for vote in voteRecordNums:
    allPrecincts[vote[:5]] = {"Precinct": vote[:5]}
allHeadToHeads = pd.DataFrame.from_dict(allPrecincts, orient = 'index')
for i in range(1, 12):
    for j in range(i, 12):
        if (i != j):
            allHeadToHeads = allHeadToHeads.join(twoCandsHeadToHead(i,j))
        elif (i == j):
            allHeadToHeads[str(i) + " over " + str(i)] = 0
            allHeadToHeads[str(i) + "-" + str(i)] = 0

In [ ]:
for i in range(1, 12):
    for j in range(1, 12):
        if(i > j):
            allHeadToHeads[str(i) + "-" + str(j)] = -allHeadToHeads[str(j) + "-" + str(i)]

In [ ]:
jointRank = {}
for i in candidateList:
    jointRank[i] = {}
    for j in candidateList:
        jointRank[i][j] = 0
for vote in voteRecord:
    ranks = voteRecord[vote]
    for candidate1 in ranks:
        for candidate2 in ranks:
            jointRank[candidate1][candidate2] += 1
pprint.pprint(jointRank, sort_dicts=False)

In [58]:
# pd.DataFrame.from_dict(jointRank, orient="index").to_csv('mutualranks.csv')

In [60]:
pprint.pprint(pairwiseVoteDifferentials([voteRecordNums, numToCandidate]))

[[0,
  7580,
  -340051,
  464572,
  411339,
  535993,
  307699,
  526493,
  488832,
  285788,
  -385695],
 [-7580,
  0,
  -87334,
  381465,
  250426,
  440386,
  253900,
  441704,
  430819,
  103843,
  -131008],
 [340051,
  87334,
  0,
  538345,
  543360,
  603613,
  437422,
  587484,
  553164,
  502558,
  -300704],
 [-464572,
  -381465,
  -538345,
  0,
  -112213,
  97650,
  -199245,
  93644,
  57463,
  -292033,
  -571177],
 [-411339,
  -250426,
  -543360,
  112213,
  0,
  214932,
  -71179,
  208160,
  167703,
  -243731,
  -578784],
 [-535993,
  -440386,
  -603613,
  -97650,
  -214932,
  0,
  -279904,
  -2938,
  -41606,
  -381012,
  -620894],
 [-307699,
  -253900,
  -437422,
  199245,
  71179,
  279904,
  0,
  279249,
  232000,
  -84900,
  -439056],
 [-526493,
  -441704,
  -587484,
  -93644,
  -208160,
  2938,
  -279249,
  0,
  -35627,
  -366144,
  -614840],
 [-488832,
  -430819,
  -553164,
  -57463,
  -167703,
  41606,
  -232000,
  35627,
  0,
  -329118,
  -559760],
 [-285788,
  -1038